Call libraries

In [2]:
import numpy as np  
import os  
import shutil  
import glob  
import random  
from datetime import datetime  
from os.path import join, basename  

from tqdm import tqdm  
import torch  
import torch.nn as nn  
import torch.nn.functional as F  
from torch.utils.data import Dataset, DataLoader  
import monai  
from segment_anything import sam_model_registry  

import argparse  
random.seed(2023)  
torch.cuda.empty_cache()  
os.environ["OMP_NUM_THREADS"] = "4"  
os.environ["OPENBLAS_NUM_THREADS"] = "4"  
os.environ["MKL_NUM_THREADS"] = "6"  
os.environ["VECLIB_MAXIMUM_THREADS"] = "4"  
os.environ["NUMEXPR_NUM_THREADS"] = "6"

Call dataset and preprocessing

In [3]:
import os
import glob
import numpy as np
import random
import torch
from os.path import basename

class Dataset:
    def __init__(self, data_root, bbox_shift=20):
        self.data_root = data_root
        self.gt_path = os.path.join(data_root, "gts")
        self.img_path = os.path.join(data_root, "imgs")
        self.gt_path_files = sorted(
            glob.glob(os.path.join(self.gt_path, "**", "*.npy"), recursive=True)
        )
        self.gt_path_files = [
            file for file in self.gt_path_files if os.path.isfile(os.path.join(self.img_path, basename(file)))
        ]
        self.bbox_shift = bbox_shift
        print(f"number of images: {len(self.gt_path_files)}")
    
    def __len__(self):
        return len(self.gt_path_files)

    def __getitem__(self, index):
        img_name = basename(self.gt_path_files[index])
        img_file_path = os.path.join(self.img_path, img_name)

        if not os.path.isfile(img_file_path):
            raise FileNotFoundError(f"Image file not found: {img_file_path}")

        img_1024 = np.load(img_file_path, allow_pickle=True)
        img_1024 = np.transpose(img_1024, (2, 0, 1))

        gt_file_path = self.gt_path_files[index]
        gt = np.load(gt_file_path, allow_pickle=True)

        label_ids = np.unique(gt)
        if len(label_ids) <= 1:
            raise ValueError(f"Ground truth contains insufficient label variations in file: {gt_file_path}")

        label_ids = label_ids[1:]  # Exclude background (assuming 0 is background)
        selected_label = random.choice(label_ids.tolist())

        gt2D = np.uint8(gt == selected_label)
        y_indices, x_indices = np.where(gt2D > 0)

        if len(x_indices) == 0 or len(y_indices) == 0:
            raise ValueError(f"No positive pixels found in ground truth for file: {gt_file_path}")

        x_min, x_max = np.min(x_indices), np.max(x_indices)
        y_min, y_max = np.min(y_indices), np.max(y_indices)
        H, W = gt2D.shape

        x_min = max(0, x_min - random.randint(0, self.bbox_shift))
        x_max = min(W, x_max + random.randint(0, self.bbox_shift))
        y_min = max(0, y_min - random.randint(0, self.bbox_shift))
        y_max = min(H, y_max + random.randint(0, self.bbox_shift))

        bboxes = np.array([x_min, y_min, x_max, y_max])

        return (
            torch.tensor(img_1024).float(),
            torch.tensor(gt2D[None, :, :]).long(),
            torch.tensor(bboxes).float(),
            img_name
        )


Dataset with configuration setup

In [4]:

task_name = "MedSAM-ViT-B"  
model_type = "vit_b"  
checkpoint = "/home/hzhb/Karry/medsam/work_dir/SAM/sam_vit_b_01ec64.pth"  
load_pretrain = Truepretrain_model_path = ""  
work_dir = "./work_dir"  
num_epochs = 500
batch_size = 2
num_workers = 0
weight_decay = 0.01
lr = 0.001
use_wandb = False
use_amp = False
resume = ""

device = "cuda:0"  

if use_wandb:  
 import wandb 
 wandb.login()  
 wandb.init(  
 project=task_name,  
 config={  
 "lr": lr,  
 "batch_size": batch_size,  
 "data_path": tr_npy_path,  
 "model_type": model_type,  
 },  
 )

Class Initialization and Forward Method

In [5]:
def __init__(self, linear_layer, m_tensor, v_tensor, block_idx, A, B, C, D, E):  
    super().__init__()  
    self.linear = linear_layer  
    self.m = m_tensor[block_idx]  
    # Frozen m with shape (3 * embed_dim)  
    self.v = v_tensor[block_idx]  
    # Frozen v with shape (3 * embed_dim, embed_dim)  
    self.A = A[block_idx]  
    # Trainable A with shape (3 * embed_dim)  
    self.B = B[block_idx]  
    # Trainable B with shape (3 * embed_dim, embed_dim)  
    self.C = C  # Trainable C with shape (embed_dim, embed_dim)  
    self.D = D  # Trainable D with shape (embed_dim, embed_dim)  
    self.E = E  # Trainable E with shape (embed_dim, embed_dim)  
    self.bias = self.linear.bias  
    self.linear.weight = None  # Remove original weight param

def forward(self, x):  
    m_new = self.m + self.A  
    B_transformed = torch.matmul(self.B, self.C)  
    B_transformed = torch.matmul(B_transformed, self.D)  
    B_transformed = torch.matmul(B_transformed, self.E)  
    v_new = self.v + B_transformed  
    weight = m_new.unsqueeze(1) * v_new  
    return F.linear(x, weight, self.bias)


pyDataset DataLoader Implementation  

In [6]:
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset

# Define your dataset class
class NpyDataset(Dataset):
    def __init__(self, img_dir, gt_dir):
        self.img_dir = img_dir
        self.gt_dir = gt_dir
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith('.npy')]
        self.gt_files = [f for f in os.listdir(gt_dir) if f.endswith('.npy')]
        self.img_files.sort()  # Ensure consistent ordering
        self.gt_files.sort()   # Ensure consistent ordering

    def __len__(self):
        # Return the length of the dataset based on the number of images
        return len(self.img_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        gt_path = os.path.join(self.gt_dir, self.gt_files[idx])
        
        # Load image and ground truth mask
        image = np.load(img_path)
        mask = np.load(gt_path)
        
        return image, mask

# Define paths and parameters
tr_npy_path = "/home/hzhb/Karry/data/眼底npy"  # Change to your training data path
train_img_path = os.path.join(tr_npy_path, "imgs")
train_gt_path = os.path.join(tr_npy_path, "gts")
batch_size = 10  # Define your desired batch size
num_workers = 4  # Adjust based on your system

# Initialize the train dataset
train_dataset = NpyDataset(train_img_path, train_gt_path)

# Create DataLoader for training dataset
train_dataloader = DataLoader(  
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=num_workers, 
    pin_memory=True
)  

# Test Dataset 
test_npy_path = os.path.join("/home/hzhb/Karry/data", "眼底npy_test")
test_img_path = os.path.join(test_npy_path, "imgs")
test_gt_path = os.path.join(test_npy_path, "gts")
test_dataset = NpyDataset(test_img_path, test_gt_path)

test_dataloader = DataLoader(  
    test_dataset, 
    batch_size=1, 
    shuffle=False, 
    num_workers=num_workers, 
    pin_memory=True
)  

print(f"Training samples: {len(train_dataset)}; Testing samples: {len(test_dataset)}")


Training samples: 20; Testing samples: 20


To check the imagee shape and mask shape

In [7]:
# Example of checking a few training samples
for i in range(5):  # Print first 5 samples
    img, mask = train_dataset[i]
    print(f"Image shape: {img.shape}, Mask shape: {mask.shape}")


Image shape: (1024, 1024, 3), Mask shape: (1024, 1024)
Image shape: (1024, 1024, 3), Mask shape: (1024, 1024)
Image shape: (1024, 1024, 3), Mask shape: (1024, 1024)
Image shape: (1024, 1024, 3), Mask shape: (1024, 1024)
Image shape: (1024, 1024, 3), Mask shape: (1024, 1024)


FCN Algorithm

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FCN(nn.Module):
    def __init__(self, num_classes):
        super(FCN, self).__init__()
        
        # Encoder (VGG-like structure)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.conv8 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, padding=1)

        # Final convolution layer
        self.final_conv = nn.Conv2d(512, num_classes, kernel_size=1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = self.pool(F.relu(self.conv7(F.relu(self.conv6(F.relu(self.conv5(x)))))))
        x = self.pool(F.relu(self.conv10(F.relu(self.conv9(F.relu(self.conv8(x)))))))
        
        # Final convolution
        x = self.final_conv(x)
        return x


In [9]:
class NpyDataset(Dataset):
    def __init__(self, npy_path):
        self.npy_path = npy_path
        self.data = self.load_data()

    def load_data(self):
        data_files = [f for f in os.listdir(self.npy_path) if f.endswith('.npy')]
        loaded_data = []
        for file in data_files:
            file_path = os.path.join(self.npy_path, file)
            loaded_data.append(np.load(file_path))
        return loaded_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx][0]  # Assuming the image is at index 0
        mask = self.data[idx][1]   # Assuming the mask is at index 1
        
        # Ensure image is float32 and mask is long (for cross-entropy loss)
        image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1)  # Change to CxHxW
        mask = torch.tensor(mask, dtype=torch.long)
        
        return image, mask


In [10]:
# import torch

# # Assuming 'images' is a torch tensor with shape [10, 1024, 1024, 3]
# print("Original shape:", images.shape)

# # Permute to get [batch, channels, height, width]
# images = images.permute(0, 3, 1, 2)

# # Verify the final shape
# print("Final shape:", images.shape)  # Expected shape: [10, 3, 1024, 1024]

# # Now pass the images to the model
# outputs = model(images)


NameError: name 'images' is not defined

In [11]:
# # Now the shape is correct: [20, 3, 1024, 1024]
# outputs = model(images)


In [12]:
import torch


# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn

class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        # Define your model architecture here

    def forward(self, x):
        # Define the forward pass here
        return x  # Modify as per your architecture

# Instantiate the model
model = FCN().to(device)  # Ensure you're moving the model to the appropriate device (GPU/CPU)


In [14]:
import os
import glob
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


Data Class Defination   

In [15]:
# Define the MyDataset class
class MyDataset(Dataset):
    def __init__(self, data_root, bbox_shift=20):
        self.data_root = data_root
        self.gt_path = os.path.join(data_root, "gts")
        self.img_path = os.path.join(data_root, "imgs")
        self.gt_path_files = sorted(
            glob.glob(os.path.join(self.gt_path, "**", "*.npy"), recursive=True)
        )
        self.gt_path_files = [
            file for file in self.gt_path_files if os.path.isfile(os.path.join(self.img_path, os.path.basename(file)))
        ]
        self.bbox_shift = bbox_shift
        print(f"Number of images: {len(self.gt_path_files)}")
    
    def __len__(self):
        return len(self.gt_path_files)

    def __getitem__(self, index):
        img_name = os.path.basename(self.gt_path_files[index])
        img_file_path = os.path.join(self.img_path, img_name)

        if not os.path.isfile(img_file_path):
            raise FileNotFoundError(f"Image file not found: {img_file_path}")

        img_1024 = np.load(img_file_path, allow_pickle=True)
        img_1024 = np.transpose(img_1024, (2, 0, 1))  # Change dimensions to CxHxW

        gt_file_path = self.gt_path_files[index]
        gt = np.load(gt_file_path, allow_pickle=True)

        label_ids = np.unique(gt)
        if len(label_ids) <= 1:
            raise ValueError(f"Ground truth contains insufficient label variations in file: {gt_file_path}")

        label_ids = label_ids[1:]  # Exclude background (assuming 0 is background)
        selected_label = np.random.choice(label_ids.tolist())

        gt2D = np.uint8(gt == selected_label)
        y_indices, x_indices = np.where(gt2D > 0)

        if len(x_indices) == 0 or len(y_indices) == 0:
            raise ValueError(f"No positive pixels found in ground truth for file: {gt_file_path}")

        x_min, x_max = np.min(x_indices), np.max(x_indices)
        y_min, y_max = np.min(y_indices), np.max(y_indices)
        H, W = gt2D.shape

        x_min = max(0, x_min - np.random.randint(0, self.bbox_shift))
        x_max = min(W, x_max + np.random.randint(0, self.bbox_shift))
        y_min = max(0, y_min - np.random.randint(0, self.bbox_shift))
        y_max = min(H, y_max + np.random.randint(0, self.bbox_shift))

        bboxes = np.array([x_min, y_min, x_max, y_max])

        return (
            torch.tensor(img_1024).float(),
            torch.tensor(gt2D[None, :, :]).long(),
            torch.tensor(bboxes).float(),
            img_name
        )


Model definition

In [17]:
import torch.nn as nn

class FCN(nn.Module):
    def __init__(self, num_classes):
        super(FCN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(64, num_classes, kernel_size=1)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.conv3(x)  # Shape: (batch_size, num_classes, height/4, width/4)
        
        # Upsample to the original size
        x = nn.functional.interpolate(x, size=(1024, 1024), mode='bilinear', align_corners=False)  # Change size accordingly
        return x  # Shape: (batch_size, num_classes, height, width)


Device Setup

In [18]:
# Set device
device = "cuda:0" if torch.cuda.is_available() else "cpu"


Model Initialization

In [19]:
# Initialize FCN model with required arguments
num_classes = 2  # Set this according to your number of classes
model = FCN(num_classes)  # Pass the num_classes parameter
model = model.to(device)  # Move model to GPU


Loss Function and Optimizer

In [20]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


Data laoder setup (defines the loss function and the optimizer used for training)

In [21]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


**DataLoader Setup (sets up the DataLoader for loading data in batches)**

In [22]:
# Set up DataLoader
train_dataset = MyDataset(data_root='/home/hzhb/Karry/data/眼底npy')
train_loader = DataLoader(
    train_dataset, 
    batch_size=2,        # Your desired batch size
    shuffle=True,        # Shuffle the data
    num_workers=0        # Number of CPU cores to load data
)


Number of images: 20


Training loop Succesful run

In [23]:
# Training loop
num_epochs = 500  # Set your desired number of epochs
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for i, data in enumerate(train_loader):
        images, labels, _, _ = data  # Unpack your data tuple
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero out gradients from previous step

        # Forward pass
        outputs = model(images)

        # Compute loss
        # Reshape labels to match the model output shape
        loss = criterion(outputs, labels.squeeze(1))  # Assuming labels are in shape (batch_size, height, width)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

    # Print average loss for the epoch
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")


Epoch [1/500], Loss: 0.4731
Epoch [2/500], Loss: 0.3551
Epoch [3/500], Loss: 0.3087
Epoch [4/500], Loss: 0.2821
Epoch [5/500], Loss: 0.2707
Epoch [6/500], Loss: 0.2643
Epoch [7/500], Loss: 0.2624
Epoch [8/500], Loss: 0.2604
Epoch [9/500], Loss: 0.2591
Epoch [10/500], Loss: 0.2597
Epoch [11/500], Loss: 0.2587
Epoch [12/500], Loss: 0.2584
Epoch [13/500], Loss: 0.2577
Epoch [14/500], Loss: 0.2577
Epoch [15/500], Loss: 0.2579
Epoch [16/500], Loss: 0.2568
Epoch [17/500], Loss: 0.2563
Epoch [18/500], Loss: 0.2562
Epoch [19/500], Loss: 0.2560
Epoch [20/500], Loss: 0.2562
Epoch [21/500], Loss: 0.2555
Epoch [22/500], Loss: 0.2557
Epoch [23/500], Loss: 0.2558
Epoch [24/500], Loss: 0.2564
Epoch [25/500], Loss: 0.2577
Epoch [26/500], Loss: 0.2555
Epoch [27/500], Loss: 0.2557
Epoch [28/500], Loss: 0.2551
Epoch [29/500], Loss: 0.2559
Epoch [30/500], Loss: 0.2549
Epoch [31/500], Loss: 0.2540
Epoch [32/500], Loss: 0.2539
Epoch [33/500], Loss: 0.2541
Epoch [34/500], Loss: 0.2540
Epoch [35/500], Loss: 0

Model save

Model Evaluation

In [97]:
import torch
from torch.utils.data import DataLoader

# Load test data
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)  # Adjust batch_size as needed
model.eval()  # Set model to evaluation mode

total_loss = 0.0
total_correct = 0
total_pixels = 0

with torch.no_grad():  # Disable gradient computation for evaluation
    for i, data in enumerate(test_loader):
        images, labels = data  # Unpack only images and labels
        images, labels = images.to(device), labels.long().to(device)

        # Forward pass
        outputs = model(images)

        # Upsample outputs if necessary
        outputs = torch.nn.functional.interpolate(outputs, size=(1024, 1024), mode='bilinear', align_corners=False)

        # Compute loss
        loss = criterion(outputs, labels)
        total_loss += loss.item()

        # Compute accuracy
        predicted = torch.argmax(outputs, dim=1)
        correct = (predicted == labels).sum().item()
        total_correct += correct
        total_pixels += labels.numel()

# Calculate average loss and accuracy
average_loss = total_loss / len(test_loader)
accuracy = total_correct / total_pixels

print(f"Average Loss: {average_loss:.4f}")
print(f"Accuracy: {accuracy * 100:.2f}%")


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [98]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [99]:
device = torch.device('cpu')


In [101]:
labels = torch.where(labels == 255, torch.tensor(1, device=labels.device), labels)


In [95]:
# Evaluation loop
total_loss = 0.0
with torch.no_grad():  # Disable gradient calculation for testing
    for images, labels in test_loader:  # Use your test_loader here
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Upsample the outputs to match the label dimensions
        outputs = nn.functional.interpolate(outputs, size=(1024, 1024), mode='bilinear', align_corners=False)

        # Compute loss (same as training)
        loss = criterion(outputs, labels.squeeze(1))  # Adjust labels if needed
        total_loss += loss.item()

        # Optionally, compute accuracy, IoU, Dice Score, etc.

# Print the total test loss
print(f'Test Loss: {total_loss / len(test_loader):.4f}')


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


**Evaluate the **model****

**Transform**

In [71]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, images_path, labels_path, transform=None):
        self.images_path = images_path
        self.labels_path = labels_path
        self.transform = transform
        self.images = sorted(os.listdir(images_path))  # List of image files
        self.labels = sorted(os.listdir(labels_path))  # List of label files

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Load image
        image = np.load(os.path.join(self.images_path, self.images[idx]))
        label = np.load(os.path.join(self.labels_path, self.labels[idx]))

        # Convert image and label to float32
        image = image.astype(np.float32)
        label = label.astype(np.long)  # Change this line to convert to Long

        if self.transform:
            image = self.transform(image)

        return image, label


**Test dataset loader**

In [72]:
import os
import numpy as np
from torch.utils.data import DataLoader

# Set paths for test dataset
test_images_path = '/home/hzhb/Karry/data/眼底npy_test/imgs'
test_labels_path = '/home/hzhb/Karry/data/眼底npy_test/gts'

# Define transformations (if needed)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create the test dataset
test_dataset = MyDataset(images_path=test_images_path, labels_path=test_labels_path, transform=transform)

# Create DataLoader for test dataset
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


**Evaluation **

In [105]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [106]:
model = FCN(num_classes=2).to(device)
model.load_state_dict(torch.load('/home/hzhb/Karry/my_trained_model.pth', map_location=device))
model.eval()


/tmp/ipykernel_7446/3226271907.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/home/hzhb/Karry/my_trained_model.pth', map_location=dev

FCN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 2, kernel_size=(1, 1), stride=(1, 1))
)

In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Load the model
model = FCN(num_classes=2).to(device)
model.load_state_dict(torch.load('/home/hzhb/Karry/my_trained_model.pth', map_location=device, weights_only=True))

model.eval()

# Evaluation metrics
total_pixels = 0
correct_pixels = 0

# Load test data
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device).float()
        labels = labels.to(device).long()

        # Forward pass
        outputs = model(images)
        outputs = F.interpolate(outputs, size=(1024, 1024), mode='bilinear', align_corners=False)

        # Get predicted class
        _, preds = torch.max(outputs, dim=1)

        # Calculate accuracy
        correct_pixels += (preds == labels).sum().item()
        total_pixels += labels.numel()

# Final accuracy
accuracy = correct_pixels / total_pixels
print(f' accuracy: {accuracy * 100:.2f}%')


NameError: name 'FCN' is not defined

In [109]:
# import matplotlib.pyplot as plt

# # Assuming you have the following data from your training loop
# epochs = list(range(1, 501))  # Epochs from 1 to 500
# loss_values = [ 
#     0.1710, 0.1703, 0.1749, 0.1716, 0.1749, 0.1712, 0.1707,
#     0.1657, 0.1654, 0.1663, 0.1651, 0.1691, 0.1672, 0.1670,
#     0.1676, 0.1712, 0.1733, 0.1843, 0.1736, 0.1734, 0.1679,
#     0.1655, 0.1664, 0.1766, 0.1676,
#     0.1417, 0.1560, 0.1605, 0.1504,
# ] + [0.1] * 500  # Replace 0.1 with actual loss values


# # Make sure the number of loss values matches the number of epochs
# if len(loss_values) != len(epochs):
#     raise ValueError("The number of loss values must match the number of epochs.")

# # Plotting the loss
# plt.figure(figsize=(12, 6))
# plt.plot(epochs[:len(loss_values)], loss_values, label='Loss', color='blue')  # Adjust epochs to match loss values
# plt.title('Training Loss over Epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.grid()
# plt.legend()
# plt.show()


ValueError: The number of loss values must match the number of epochs.